In [ ]:
import pandas as pd
pbr_excel_path = r"C:\Users\Andyfish\.conda\envs\fs_env\codes\data\PBR_201412_202409.xlsx"
df_pbr= pd.read_excel(pbr_excel_path)
df_fsdata = pd.read_excel(r"C:\Users\Andyfish\.conda\envs\fs_env\codes\data\fscore_data_fi3.xlsx")

In [ ]:
import backtrader as bt
import pandas as pd
# import yfinance as yf   
from datetime import datetime
import warnings
import fscore_pool
import logging
from importlib import reload

reload(fscore_pool)
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

#Define(2015,3,30 - 2020,2,7)(2020,2,10 - 2024,12,13)
ts = 2
if ts == 0: #10y
    start_date = datetime(2015,3,30)
    end_date = datetime(2024,12,13)
elif ts == 1:#IS
    start_date = datetime(2015,3,30)
    end_date = datetime(2020,2,7)
elif ts == 2:#OS
    start_date = datetime(2020,2,10)
    end_date = datetime(2025,9,19)
is_rebalancing = True
time_period = None

class f_score_trading(bt.Strategy):
    params =  (("is_log",True),("poolsize",0),("rebalancing",False),("dfsp",pd.DataFrame()),("ca",10000000.0),("wr",1))

    def log(self,txt,dt=None):
        if self.params.is_log:
            dt = dt or self.datas[0].datetime.date()
            print(f"{dt}\t{txt}")

    #初始條件
    def __init__(self):
        self.order = None
        self.status = 0
        self.closep = dict()
        self.rebalance_date = []
        self.p_sotcks = 0

        print("Pool size:\t",self.params.poolsize)
        print(f"Rebalancing: {self.params.rebalancing}")
        
        # print(f"dfsp: {self.params.dfsp.shape},{self.params.dfsp}")
        
        for i,d in enumerate(self.datas):
            self.closep[d] = d.close

    #每bar交易執行
    def next(self):
        #Date for everyday
        dt = self.datetime.date()
        list_buying_target = []
        self.p_sotcks = 0

        # print(f"Today:{self.datetime.date()}")

        if self.order:  
            return

        #無調整，純買進
        if self.status == 0 and self.params.rebalancing == False:
            for i,d in enumerate(self.datas):
                sid = d._name
                pos = self.getposition(d).size

                # self.log(f"{sid},close={self.closep[d][0]} ")
                # print(f"{dt},{sid},pos={pos}")

                if cerebro.broker.getcommissioninfo(d) != 0.001425 : cerebro.broker.setcommission(commission=0.001425) #買進費率
                self.order_target_percent(data=d,target=1/self.params.poolsize)
                logging.info(f"Buy {sid}, position:{pos}")
            self.status = 1

        #含調整 賣出，買進
        if (self.status == 1) and (self.params.rebalancing == True) and (dt in self.params.dfsp.loc[:,"ymd"].to_list()) :
            logging.info(f"Rebalancing_adjust: {dt}")
            #
            list_buying_target = self.params.dfsp.loc[self.params.dfsp.loc[:,"ymd"]==dt].loc[:,"pool_list"].to_list()[0]
            list_buying_target = [i+".TW" for i in list_buying_target]
            original_len = len(list_buying_target)
            logging.info(f"Target: {list_buying_target}")

            #算出優先權重股數********
            for d in self.datas:
                if (self.getposition(d).size !=0) and (d._name in list_buying_target) : self.p_sotcks += 1
            total_weights = self.params.wr * self.p_sotcks + 1 * (original_len - self.p_sotcks)
            p_weight = self.params.wr / total_weights
            other_weight = 1 / total_weights

            #不在欲買清單就Close
            for d in self.datas:
                #若已買，不在欲買其中就刪掉
                if (self.getposition(d).size !=0) and (not d._name in list_buying_target):
                    if cerebro.broker.getcommissioninfo(d) != 0.001425+0.003 : cerebro.broker.setcommission(commission=0.001425+0.003) #賣出費率
                    self.close(d)
                    logging.info(f"Closed {d._name}, at position:{self.getposition(d).size}")
                #若已買，在欲買其中就不再重買
                elif (self.getposition(d).size !=0) and (d._name in list_buying_target):
                    if cerebro.broker.getcommissioninfo(d) != 0.001425 : cerebro.broker.setcommission(commission=0.001425) #買進費率
                    self.order_target_percent(data=d,target=p_weight)
                    list_buying_target.remove(d._name)
                #re-Buy
                if (self.getposition(d).size ==0) and (d._name in list_buying_target):
                    sid = d._name
                    pos = self.getposition(d).size

                    if cerebro.broker.getcommissioninfo(d) != 0.001425 : cerebro.broker.setcommission(commission=0.001425) #買進費率
                    self.order_target_percent(data=d,target=other_weight)
                    logging.info(f"Buy {sid}, position:{pos}")
 
        #含調整 買進首期
        if (self.status == 0) and (self.params.rebalancing == True) and (dt in self.params.dfsp.loc[:,"ymd"].to_list()) :
            logging.info(f"Rebalancing_first_buy: {dt}")
            #
            list_buying_target = self.params.dfsp.loc[self.params.dfsp.loc[:,"ymd"]==dt].loc[:,"pool_list"].to_list()[0]
            list_buying_target = [i+".TW" for i in list_buying_target]
            for d in self.datas:
                if d._name in list_buying_target:
                    sid = d._name
                    pos = self.getposition(d).size

                    self.order_target_value(data=d,target=self.params.ca/len(list_buying_target))
                    logging.info(f"Buy {sid}, position:{pos}")
            self.status = 1
        
#Initialize
cerebro = bt.Cerebro()

#files and datas
#不調整，只載入首期pool
# pbr_excel_path = r"C:\Users\Andyfish\.conda\envs\fs_env\codes\data\PBR_201409_202406.xlsx"
# df_pbr= pd.read_excel(pbr_excel_path)
# df_fsdata = pd.read_excel(r"C:\Users\Andyfish\.conda\envs\fs_env\codes\data\fscore_data_new.xlsx")
df_sep_pool_signal = pd.DataFrame()
pool = []

ava_ym_list = sorted(list(set(df_pbr["ym"].astype("str"))))
ava_ym_list = [datetime.strptime(i+"15","%Y%m%d") for i in ava_ym_list]
mask_ym = [i <= start_date for i in ava_ym_list]
latest_ym_int = int([ava_ym_list[i] for i in range(len(ava_ym_list)) if mask_ym[i]==True][-1].strftime("%Y%m%d")[0:-2])
if not is_rebalancing:
    p_pool = fscore_pool.get_pool(latest_ym_int,pbr_raw=df_pbr,fsdata=df_fsdata)
    pool = p_pool["sid"].to_list()
    pool = [i+".TW" for i in pool]
    if "1524.TW" in pool : pool.remove("1524.TW")
    for code in pool:
        stock_df = fscore_pool.updating_10y(code)
        data = bt.feeds.PandasData(dataname=stock_df, fromdate=start_date, todate=end_date) 
        cerebro.adddata(data,name=str(code))
    # data = bt.feeds.PandasData(dataname=fscore_pool.updating_10y("TWII"), fromdate=start_date, todate=end_date) 
    # cerebro.adddata(data,name="TWII")
    # pool = ["TWII"]

#如果考慮每季平倉調整，載入bigpool
else:
    # df_sep_pool_signal,list_bigpool = pd.read_pickle(r"C:\Users\Andyfish\.conda\envs\fs_env\codes\data\signal_pool.pickle"),pd.read_pickle(r"C:\Users\Andyfish\.conda\envs\fs_env\codes\data\bigpool.pickle").loc[:,0].to_list() #預載pickle檔，可替代成get sep pool
    df_sep_pool_signal,list_bigpool = fscore_pool.get_sep_pool(timeperiod=time_period,is_save=True,fsdata=df_fsdata,pbdata=df_pbr)
    df_sep_pool_signal.loc[:,"ymd"] = [i.date() for i in df_sep_pool_signal.loc[:,"ymd"]]
    
    list_bigpool = [i+".TW" for i in list_bigpool]
    for code in list_bigpool:
        stock_df = fscore_pool.updating_10y(code)
        data = bt.feeds.PandasData(dataname=stock_df, fromdate=start_date, todate=end_date) 
        cerebro.adddata(data,name=str(code))

#Strategies
cash = 10000000.0
cerebro.addstrategy(f_score_trading,is_log=True,poolsize=len(pool),rebalancing=is_rebalancing,dfsp=df_sep_pool_signal,ca=cash)

#Settings
cerebro.broker.setcash(cash)
cerebro.broker.setcommission(commission=0.001425)
cerebro.addanalyzer(bt.analyzers.PyFolio,_name="pf")

#Port. Value
initial_value = cerebro.broker.getvalue()
logging.info('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
end_value = cerebro.broker.getvalue()
logging.info('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
rr = (end_value - initial_value) / initial_value
print(f"累積報酬: {rr*100:.4f} %")
# cerebro.plot(iplot=True)[0][0]

In [ ]:
#大盤數據+時間處理
logger.setLevel(logging.WARNING)
df_twii = fscore_pool.updating_10y("TWII")
df_twii["TWSE index"] = df_twii["Close"].pct_change(1)

df_twii.reset_index(drop=False,inplace=True)
mask = (df_twii["Date"] > start_date) & (df_twii["Date"] <= end_date)
df_twii = df_twii.loc[mask]
df_twii.set_index("Date",inplace=True)
df_twii.index = df_twii.index.tz_localize(tz="UTC")

strades = results[0]
pf = strades.analyzers.getbyname('pf')
returns, positions, transactions, gross_lev = pf.get_pf_items()
bmrt = df_twii['TWSE index']

import empyrical
import scipy.stats
print(f"{empyrical.annual_return(returns)*100:.4f}")
print(f"{empyrical.cum_returns(returns).iloc[-1]*100:.4f}")
print(f"{empyrical.sharpe_ratio(returns):.4f}")
print(f"{empyrical.max_drawdown(returns)*100:.4f}")
print(f"{empyrical.sortino_ratio(returns):.4f}")
print(f"{scipy.stats.skew(returns):.4f}")
print(f"{scipy.stats.kurtosis(returns):.4f}\n")
# print(f"{empyrical.alpha(returns,factor_returns=df_twii['TWSE index']):.4f}")
# print(f"{empyrical.beta(returns,factor_returns=df_twii['TWSE index']):.4f}\n")

print(f"{empyrical.annual_return(bmrt)*100:.4f}")
print(f"{empyrical.cum_returns(bmrt).iloc[-1]*100:.4f}")
print(f"{empyrical.sharpe_ratio(bmrt):.4f}")
print(f"{empyrical.max_drawdown(bmrt)*100:.4f}")
print(f"{empyrical.sortino_ratio(bmrt):.4f}")
print(f"{scipy.stats.skew(bmrt):.4f}")
print(f"{scipy.stats.kurtosis(bmrt):.4f}")
# print(f"{empyrical.alpha(returns,factor_returns=df_twii['TWSE index']):.4f}")
# print(f"{empyrical.beta(returns,factor_returns=df_twii['TWSE index']):.4f}")


In [ ]:
# import pyautogui,time,re
# import pandas as pd

# time.sleep(10)
# print("Start")
# for i in list_bigpool:
#     pyautogui.doubleClick()
#     pyautogui.press("backspace")
#     pyautogui.typewrite(i)
#     # time.sleep(0.01)
#     pyautogui.press("enter")

In [ ]:
#大盤數據+時間處理
df_twii = fscore_pool.updating_10y("TWII")
df_twii["TWSE index"] = df_twii["Close"].pct_change(1)

df_twii.reset_index(drop=False,inplace=True)
mask = (df_twii["Date"] > start_date) & (df_twii["Date"] <= end_date)
df_twii = df_twii.loc[mask]
df_twii.set_index("Date",inplace=True)
df_twii.index = df_twii.index.tz_localize(tz="UTC")

%matplotlib inline

strades = results[0]
pf = strades.analyzers.getbyname('pf')
returns, positions, transactions, gross_lev = pf.get_pf_items()

import pyfolio as pyf
pyf.create_full_tear_sheet(returns=returns,positions=positions,transactions=transactions,benchmark_rets=df_twii["TWSE index"])

In [ ]:
# from empyrical import alpha_beta
# print(alpha_beta(returns,df_twii["TWSE index"]))

In [ ]:
#用zlenv0執行

# returns.index = returns.index.date
# df_twii.index = df_twii.index.date
# print(type(returns.index))
# print(type(df_twii.index))
# returns.index = pd.to_datetime(returns.index)
# df_twii.index = pd.to_datetime(df_twii.index)

# import quantstats as qs
# qs.reports.html(returns=returns,benchmark=df_twii["TWSE index"],output='F_score_hold_10y.html',title="F_score_hold_10y") 